# Links
- https://www.kaggle.com/c/otto-group-product-classification-challenge

# Solutions
- 4-th place: https://github.com/diefimov/otto_2015
- 12-th place: https://github.com/tks0123456789/kaggle-Otto
- 66-th place: https://github.com/ahara/kaggle_otto

In [2]:
import pandas as pd
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import cross_val_score
import numpy as np

In [1]:
%run ds_tools/dstools/ml/xgboost_tools.py

/Users/dmitri/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
def cv_test(est):
    scores = cross_val_score(estimator=est, X=features, y=labels, cv=3, scoring='log_loss')
    print('mean: {mean}, std: {std}'.format(mean=scores.mean(), std=scores.std()))

In [6]:
def submission(est):
    model = est.fit(features, labels)

    df_test = read_csv('test.csv.gz', index_col='id')

    y_pred = model.predict_proba(df_test)

    res_df = DataFrame(y_pred, columns=['Class_%d' % n for n in range(1, 10)], index=df_test.index)
    res_df.to_csv('results.csv', index_label='id')

In [5]:
df = pd.read_csv('train.csv.gz', index_col='id')

features = df.drop(['target'], axis=1)
labels = df.target.apply(lambda e: e[6:]).astype(np.int16)-1

In [7]:
xgb_params = {
    "objective": "multi:softprob",
    "num_class": 9,
    "eta": 0.1,
    "num_rounds": 10000,
    "max_depth": 10,
    "min_child_weight": 4,
    "gamma": 1,
    "subsample": 0.7,
    "colsample_bytree": 0.7,
    "scale_pos_weight": 1,
    "silent": 2,
    "eval_metric": "mlogloss",
}

# mean: -0.4829334319, std: 0.00535488081374
# cv execution time: 772.511557102 sec
est1 = XGBoostClassifier(**xgb_params)

In [8]:
# mean: -0.47376124206, std: 0.00323548514227
# cv execution time: 3183.97259498 sec
est2 = CalibratedClassifierCV(base_estimator=est1, cv=5, method='isotonic')